In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def load_data(root='data/CroppedYaleB', reduce=4):
    """ 
    Load ORL (or Extended YaleB) dataset to numpy array.
    
    Args:
        root: path to dataset.
        reduce: scale factor for zooming out images.
        
    """ 
    images, labels = [], []

    for i, person in enumerate(sorted(os.listdir(root))):
        
        if not os.path.isdir(os.path.join(root, person)):
            continue
        
        for fname in os.listdir(os.path.join(root, person)):    
            
            # Remove background images in Extended YaleB dataset.
            if fname.endswith('Ambient.pgm'):
                continue
            
            if not fname.endswith('.pgm'):
                continue
                
            # load image.
            img = Image.open(os.path.join(root, person, fname))
            img = img.convert('L') # grey image.

            # reduce computation complexity.
            img = img.resize([s//reduce for s in img.size])

            # TODO: preprocessing.

            # convert image to numpy array.
            img = np.asarray(img).reshape((-1,1))

            # collect data and label.
            images.append(img)
            labels.append(i)

    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)

    return images, labels

In [ ]:
# Load ORL dataset.
X, Y = load_data(root='data/ORL', reduce=2)
print('ORL dataset: X.shape = {}, Y.shape = {}'.format(X.shape, Y.shape))

## Euclidean multiplicative update alogrithm

In [8]:
"""
https://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf
V = WH

W = (n x r) matrix
H = (r x m) matrix

cost function SSE = sum(V-WH)**2

multiplicative update rules
 H = H*(W.T@V)/(W.T@W@H)
 W = W(V@H.T)/(W@H@HT)
 
 check error
 
 itterate

"""

def euclidean_multiplicative_update (V, r, iteration):
    
    loss = []
    W = np.random.rand(V.shape[0],r)*255
    H = np.random.rand(r,V.shape[1])*255
    
    for i in range(iteration):
        H = H*(W.T@V)/(W.T@W@H)
        W = W*(V@H.T)/(W@H@H.T)
        l = np.sum((V-W@H)**2)
        loss.append(l)
        
    return W, H, loss        


In [10]:
W, H, loss  = euclidean_multiplicative_update(X, 10, 100)

In [11]:
print(len(loss))
plt.plot(range(0,len(loss)),loss)

100


In [23]:
'''
https://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf
implimentation from page 3 formula 5
'''

def KL_divergence_update(V,r,iteration):

    loss = []
    W = np.random.rand(V.shape[0],r)*255
    H = np.random.rand(r,V.shape[1])*255
    
    for i in range(iteration):
        H = H * np.dot(W.T,V / np.dot(W,H)) / np.sum(W,0)[:,None]
        W = W * np.dot(V / np.dot(W,H),H.T) / np.sum(H,1)
        l=np.sum(V*np.log((V++1e-10)/(W@H))-V+(W@H))
        loss.append(l)
        
    return W, H, loss 
    

In [24]:
W, H, loss  = KL_divergence_update(X, 10, 50)

In [ ]:
plt.plot(range(0,len(loss)),loss)